In [2]:
import torch
import torch.nn.functional as F
import numpy as np

In [16]:
import re
mystr = 'asd23dsa'
mystr = list(filter(lambda x: re.match(r"[a-zA-Z']", x), mystr))
print(mystr)

['a', 's', 'd', 'd', 's', 'a']


In [5]:
labels =[
  "#",
  "'",
  "A",
  "B",
  "C",
  "D",
  "E",
  "F",
  "G",
  "H",
  "I",
  "J",
  "K",
  "L",
  "M",
  "N",
  "O",
  "P",
  "Q",
  "R",
  "S",
  "T",
  "U",
  "V",
  "W",
  "X",
  "Y",
  "Z",
  " "
]
labels_map = {ch:idx for idx, ch in enumerate(labels)}
print(labels_map)

{'#': 0, "'": 1, 'A': 2, 'B': 3, 'C': 4, 'D': 5, 'E': 6, 'F': 7, 'G': 8, 'H': 9, 'I': 10, 'J': 11, 'K': 12, 'L': 13, 'M': 14, 'N': 15, 'O': 16, 'P': 17, 'Q': 18, 'R': 19, 'S': 20, 'T': 21, 'U': 22, 'V': 23, 'W': 24, 'X': 25, 'Y': 26, 'Z': 27, ' ': 28}


In [17]:
def cal(target, prob):
    new_target = '#'
    for ch in target:
        new_target += ch + '#'
    print(new_target)
    
    def get_idx(ch):
        return labels_map[ch]
    T = prob.size(0)
    N = len(new_target)
    f = torch.zeros(T, N)
    f[0][0] = prob[0][0]
    f[0][get_idx(new_target[0])] = prob[0][get_idx(new_target[0])]
    for i in range(1, T):
        for j in range(len(new_target)):
            c = get_idx(new_target[j])
            val = f[i-1][j]
            if j > 0:
                val += f[i-1][j-1]
            if j > 1 and new_target[j] != '#' and new_target[j] != new_target[j-2]:
                val += f[i-1][j-2]
            f[i][j] = val * prob[i][c]
        sum = 0
        for j in range(len(new_target)):
            sum += f[i-1][j]
        for j in range(len(new_target)):
            f[i-1][j] /= sum
    ans = (f[T-1][-1] + f[T-1][-2]).log_()        
    return ans
target = 'AB'
prob = torch.FloatTensor([[0.2, 0.2, 0.1, 0.5],[0.1, 0.1, 0.7, 0.1],[0.3, 0.1, 0.4, 0.2]])
print(f'without softmax:{cal(target, prob)}')
prob = F.softmax(prob, dim=1)
print(f'with softmax:{cal(target, prob)}')

#A#B#
without softmax:-1.5994876623153687
#A#B#
with softmax:-1.4478483200073242


In [25]:
prob = torch.FloatTensor([[0.1, 0.1, 0.7, 0.1],[0.1, 0.1, 0.1, 0.7],[0.1, 0.1, 0.1, 0.7]])
prob = F.log_softmax(prob, dim=1).view(3, 1, 4)
target = torch.IntTensor([2, 3])
input_lengths = torch.IntTensor([3])
target_lengths = torch.IntTensor([2])
val = F.ctc_loss(log_probs=prob, targets= target, input_lengths=input_lengths, target_lengths=target_lengths, reduction='none')
print(f'torch.ctc_loss: {val}')

torch.ctc_loss: tensor([1.8386])


In [ ]:
transcript = 'HELLO WORLD'
print(list(transcript))
transcript = list(filter(None, [labels_map.get(x) for x in list(transcript)]))
print(transcript)

In [ ]:
'a' + 1